In [ ]:
!pip install -q --upgrade torch torchvision torchaudio

In [ ]:
%load_ext autoreload
%autoreload 2

import os,sys
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../../.."))
sys.path.append(os.path.abspath("../.."))
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install -q -r ../../requirements.txt
!pip install -q lightning==2.5.0

In [ ]:
import os
import dotenv
dotenv.load_dotenv("../../.env")
DB_MINIO_BUCKET = os.environ["DB_MINIO_BUCKET"]

In [ ]:
from workflow.dags.extract import extract_data
from training.dlrm.dlrm import train_dlrm
from movie_recommender.common.workflow import StorageClient
from training.data import preprocess_data

In [ ]:
dir = extract_data(os.environ["DATABASE_URL"])
StorageClient.get_instance().upload_folder_to_bucket(dir, DB_MINIO_BUCKET)

In [ ]:
ratings, movies = StorageClient.get_instance().download_parquet_from_bucket(
    DB_MINIO_BUCKET, "ratings", "movies")
train, test = preprocess_data(ratings,movies, max_rating=5, train_size=.9)

In [ ]:
import pandas as pd
from movie_recommender.common.utils import user_cols, movie_cols


def split_(df: pd.DataFrame):
    movies = df[movie_cols(df)].drop_duplicates("movie_id").set_index("movie_id")
    users = df[user_cols(df)].drop_duplicates("user_id").set_index("user_id")
    return users, movies


users_features, movies_features = split_(pd.concat([train, test], axis=0))
StorageClient.get_instance().upload_parquet_to_bucket(
    DB_MINIO_BUCKET,
    users_features=users_features,
    movies_features=movies_features,
)

In [ ]:
train.head(1)

In [ ]:
train_dlrm(
    train,
    test,
    int(os.environ["EPOCH"]),
    "movie_recom",
    batch_size=64*4*2
)

In [ ]:
# test_dlrm_model()